<a href="https://colab.research.google.com/github/abhatia903/Concept_Drift_Data/blob/main/TrainByUsingAdwin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
%matplotlib inline
import math
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/updateganga_data.csv')
print('Number of rows and columns:', df.shape)
df.head(5)

Number of rows and columns: (46983, 10)


,Unnamed: 0,Date,Frame No,BAT,COND,DO,ORP,PH,WT,Target
0,0,12-01-19 12:57,0,91.0000,0.0447,14.8328,0.0848,8.8797,22.6502,NODRIFT
1,1,12-01-19 12:58,1,92.0000,0.0447,14.1703,0.0863,8.8668,22.5670,NODRIFT
2,2,12-01-19 14:54,0,90.0000,46.1952,13.5012,0.1429,12.3599,18.4008,NODRIFT
3,3,12-01-19 14:55,1,91.0000,52.6097,20.7529,0.1462,12.8257,18.0136,NODRIFT
4,4,12-01-19 14:56,2,91.0000,52.3998,12.4094,0.1467,12.9990,18.1846,NODRIFT


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
df['Target']= label_encoder.fit_transform(df['Target'])
 
df['Target'].unique()
df[df['Target']==0]

,Unnamed: 0,Date,Frame No,BAT,COND,DO,ORP,PH,WT,Target
25,25,12-01-19 15:17,23,90.0000,53.2876,8.3976,0.1441,13.1781,17.9444,0
57,57,12-01-19 15:50,55,90.0000,55.4216,17.5996,0.1555,13.2321,17.7470,0
83,83,30-03-19 10:36,11,97.0000,1.0197,108.8841,0.0878,9.9153,29.0336,0
89,89,30-03-19 10:42,17,95.0000,1.0200,108.9132,0.0873,9.8432,28.6580,0
95,95,30-03-19 12:28,5,85.0000,392.6326,108.9229,0.0292,14.2488,29.2475,0
...,...,...,...,...,...,...,...,...,...,...
38239,38239,31-12-19 5:26,180,83.0000,396.6415,19.2422,0.1113,8.0206,16.0129,0
38303,38303,31-12-19 6:31,244,84.0000,396.6355,19.2422,0.1106,8.0024,15.9840,0
38431,38431,31-12-19 8:43,117,84.0000,396.8191,19.2422,0.1102,8.0964,15.9387,0
38471,38471,31-12-19 9:23,157,83.0000,396.8784,19.2422,0.1100,8.1219,15.9211,0


In [6]:
main_data=df.iloc[:,[4,5,7,8]]
main_data

,COND,DO,PH,WT
0,0.0447,14.8328,8.8797,22.6502
1,0.0447,14.1703,8.8668,22.5670
2,46.1952,13.5012,12.3599,18.4008
3,52.6097,20.7529,12.8257,18.0136
4,52.3998,12.4094,12.9990,18.1846
...,...,...,...,...
46978,721.8673,10.0086,9.3776,27.1307
46979,722.1170,10.0086,9.3750,27.1423
46980,722.0287,10.0086,9.3758,27.1400
46981,721.9607,10.0086,9.3802,27.1350


In [7]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(main_data)

In [9]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
# create dataset
X, y = dataset,df['Target']
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(23491, 4) (23492, 4) (23491,) (23492,)


In [10]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))



In [11]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [12]:
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer='adam', metrics=METRICS)

history = model.fit(X_train, y_train, epochs=30, batch_size=70, validation_data=(X_test, y_test), verbose=1)

model.summary()

Epoch 1/30
336/336 [==============================] - 11s 18ms/step - loss: 0.1964 - tp: 23073.0000 - fp: 291.0000 - tn: 0.0000e+00 - fn: 127.0000 - accuracy: 0.9822 - precision: 0.9875 - recall: 0.9945 - auc: 0.5194 - prc: 0.9881 - val_loss: 0.0774 - val_tp: 23163.0000 - val_fp: 329.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.9860 - val_precision: 0.9860 - val_recall: 1.0000 - val_auc: 0.4338 - val_prc: 0.9835
Epoch 2/30
336/336 [==============================] - 4s 11ms/step - loss: 0.0687 - tp: 23200.0000 - fp: 291.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - accuracy: 0.9876 - precision: 0.9876 - recall: 1.0000 - auc: 0.5173 - prc: 0.9880 - val_loss: 0.0760 - val_tp: 23163.0000 - val_fp: 329.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.9860 - val_precision: 0.9860 - val_recall: 1.0000 - val_auc: 0.5131 - val_prc: 0.9860
Epoch 3/30
336/336 [==============================] - 6s 17ms/step - loss: 0.0676 - tp: 23200.0000 - fp: 291.0000 - tn: 0.0000e

In [ ]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
train_predict.shape

(23491, 1)

In [13]:
loss,tp,fp,tn,fn,accuracy,precision, recall,auc,prc = model.evaluate(X_test, y_test, verbose=0)

In [14]:
loss

0.07265566289424896

In [15]:
tp,fp,tn,fn,accuracy,precision,recall,auc,prc

(23163.0,
 329.0,
 0.0,
 0.0,
 0.9859952330589294,
 0.9859952330589294,
 1.0,
 0.6094151139259338,
 0.9904379844665527)

In [16]:
precision

0.9859952330589294

In [17]:
recall 

1.0

In [18]:
f1score=(2*precision*recall)/(precision+recall)
f1score

0.9929482373834807

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['acc'], label='Train Loss')
plt.plot(history.history['val_acc'], label='Test Loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show();

KeyError: ignored

<Figure size 576x288 with 0 Axes>

In [ ]:
y_test

In [ ]:
aa=[x for x in range(23492)]
plt.figure(figsize=(8,4))
plt.plot(aa, y_test[:23492], marker='.', label="actual")
plt.plot(aa, test_predict[:,0][:23492], 'r', label="prediction")
plt.tick_params(left=False, labelleft=True) #remove ticks
plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show()

In [19]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [22]:
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X,y)



Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [23]:
pred=clf.predict(X)

In [25]:
from sklearn.metrics import accuracy_score

In [28]:
score=accuracy_score(y,pred)

In [29]:
score

0.9868037375220825